In [1]:
import tensorflow as tf

print(tf.__version__)


2024-04-06 06:58:55.129010: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2.16.1


In [2]:
import tensorflow as tf
print("TensorFlow version: ", tf.__version__)

from tensorflow.python.compiler.tensorrt import trt_convert as trt
from tensorflow.python.saved_model import tag_constants

TensorFlow version:  2.16.1


In [3]:
SAVED_MODEL_DIR =  "/saved_model/resnet_v1_fp32_savedmodel"

In [4]:
conversion_params = trt.DEFAULT_TRT_CONVERSION_PARAMS._replace(
    precision_mode=trt.TrtPrecisionMode.FP32)

converter = trt.TrtGraphConverterV2(
    input_saved_model_dir=SAVED_MODEL_DIR,
conversion_params=conversion_params)

Instructions for updating:
Use individual converter parameters instead
INFO:tensorflow:Linked TensorRT version: (8, 6, 1)
INFO:tensorflow:Loaded TensorRT version: (8, 6, 1)


In [6]:
import tensorflow as tf

# Fashion MNIST 데이터셋 로드
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

# 데이터 전처리
x_train, x_test = x_train / 255.0, x_test / 255.0
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

# 모델 생성
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10)
])

# 모델 컴파일
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# 모델 학습
model.fit(x_train, y_train, epochs=5)

# 모델 저장
tf.saved_model.save(model, 'fashion_mnist_model')



Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.8056 - loss: 0.5541
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8957 - loss: 0.2902
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9164 - loss: 0.2308
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9269 - loss: 0.2014
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9363 - loss: 0.1729
INFO:tensorflow:Assets written to: fashion_mnist_model/assets


INFO:tensorflow:Assets written to: fashion_mnist_model/assets


In [8]:
! python -m tf2onnx.convert --saved-model fashion_mnist_model --output model.onnx


2024-04-06 07:00:27.176142: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
<frozen runpy>:128: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
2024-04-06 07:00:28,361 - WARNING - ***IMPORTANT*** Installed protobuf is not cpp accelerated. Conversion will be extremely slow. See https://github.com/onnx/tensorflow-onnx/issues/1557
2024-04-06 07:00:28.402424: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-06 07:00:28.405665: I external/local_xla/xla/stream_executor/cu

In [10]:
import tensorrt as trt

TRT_LOGGER = trt.Logger(trt.Logger.WARNING)

def build_engine(onnx_file_path):
    builder = trt.Builder(TRT_LOGGER)
    network = builder.create_network(1 << int(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH))
    parser = trt.OnnxParser(network, TRT_LOGGER)

    # ONNX 파일 로드
    with open(onnx_file_path, 'rb') as model:
        parser.parse(model.read())

    # 최적화된 엔진 빌드
    config = builder.create_builder_config()
    config.max_workspace_size = 8 << 30  # 1GB
    return builder.build_engine(network, config)

engine = build_engine('model.onnx')


[04/06/2024-07:01:40] [TRT] [W] onnx2trt_utils.cpp:374: Your ONNX model has been generated with INT64 weights, while TensorRT does not natively support INT64. Attempting to cast down to INT32.
[04/06/2024-07:01:40] [TRT] [E] 4: [network.cpp::validate::3163] Error Code 4: Internal Error (Network has dynamic or shape inputs, but no optimization profile has been defined.)


/tmp/ipykernel_2548/3480550874.py:16: DeprecationWarning: Use set_memory_pool_limit instead.
  config.max_workspace_size = 8 << 30  # 1GB
/tmp/ipykernel_2548/3480550874.py:17: DeprecationWarning: Use build_serialized_network instead.
  return builder.build_engine(network, config)
